<a href="https://colab.research.google.com/github/SeongwonTak/TIL_swtak/blob/master/Pytorch/DL_with_pytorch_ch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DL_with_pytorch ch2
pytorch 패키지를 공부해본다.
밑딥과 병행하며 보는 책 내용을 정리.

출처 : Pytorch로 시작하는 딥러닝

In [2]:
import torch

## 기본 구조와 데이터 표현

추상적 관점에서 신경망 구현 코드의 기본 구조를 살펴보자.

In [ ]:
x, y = get_data()  # x - 훈련 데이터,  y - 목표변수
w, b = get_weights()  # w - 가중치,  b - 편향
for i in range(50000): 
  y_pred = simple_network(x)  # 학습
  loss = loss_fn(y, y_pred)  # 오차정의 (MSE 등을 사용한다.)
  if i % 5000 == 0:  # 1에폭당 오차계산.
    print(loss)  
  optimize(learning_rate)

Pytorch는 Tensor 형태로 데이터를 표현할 수 있다.

- 0차원 스칼라
- 1차원 벡터
- 2차원 행렬
- 3차원 텐서, 4차원 텐서....


In [3]:
# 스칼라
x = torch.rand(10)
print(x)
print(x.size())

tensor([0.4026, 0.1699, 0.7372, 0.0516, 0.8817, 0.7806, 0.3172, 0.9404, 0.4037,
        0.5913])
torch.Size([10])


In [4]:
# 벡터
vec = torch.FloatTensor([1.1, 2.5, 3.14, 4.0])
vec.size()

torch.Size([4])

In [6]:
# 행렬
boston_tensor = torch.from_numpy(boston.data)
boston_tensor.size()

NameError: ignored

3차원 이상도 표현 가능하다.
3차원의 경우는 (이미지의 높이, 이미지의 폭, RGB)를 나타내는 형태의 이미지 표현으로 많이 쓰인다.

여기서 4차원으로 넘어가는 경우는 batch처리를 진행한다.

## GPU 지원 Tensor

In [9]:
a = torch.rand(2, 2)
b = torch.rand(2, 2)
c = a + b
d = torch.add(a, b)
print(a)
print(b)
print(c)
print(d)

tensor([[0.6009, 0.8330],
        [0.0619, 0.0770]])
tensor([[0.4097, 0.6336],
        [0.3323, 0.9605]])
tensor([[1.0106, 1.4667],
        [0.3942, 1.0375]])
tensor([[1.0106, 1.4667],
        [0.3942, 1.0375]])


In [ ]:
a = torch.rand(10000, 10000)
b = torch.rand(10000, 10000)
a.matmul(b)

a = a.cuda()
b = b.cuda()

a.matmul(b)

여기서 cuda란?

## Variable
변수 표현은 '계산 그래프'으로 표현된다.
즉
- tensor 객체
- 기울기 객체
- 변수를 생성하는 함수의 참조

In [ ]:
# Variable 객체의 예시
x = Variable(torch.ones(2,2),requires_grad=True)
y = x.mean()

y.barkward()

x.grad
x.data


## 데이터 생성하기
get_data를 통해 x와 y를 생성한다.

In [12]:
def get_data():
  train_X = np.asarray([1.1, 2.2, 5, 3, 4, 4.8])
  train_Y = np.assarry([5, 11.3, 106.24, 7.2, 1.88, 10.15])
  dtype = torch.FloatTensor

  X = Variable(torch.from_numpy(train_X).type(dtype), requires_grad = False).view(6, 1)
  y = Variable(torch.from_numpy(train_Y).type(dtype), requires_grad = False)

  return X, y

가중치 또한 생성 가능하다.

In [13]:
def get_weights():
  w = Variable(torch.randn(1), requires_grad = True)
  b = Variable(torch.randn(1), requires_grad = True)

위의 두 가지를 바탕으로 네트워크를 구현할 수 있다.
가중치와 x의 행렬 곱에 편향 b를 합치는 예시를 생각하자.

In [14]:
def simple_network(x):
  y_pred = torch.matmul(x, w) + b
  return y_pred

# 참고. nn.Linear를 활용하여 간략하게 표현 가능.

모델 생성의 결과를 오차함수로 확인해보자.

In [ ]:
def loss_fn(y, y_pred):
  loss = (y_pred - y).pow(2).sum()
  for param in [w, b]:  # w와 b의 변화 정도를 기울기로 계산.
    if not param.grad is None: param.grad.data.zero_()
  loss.backward()
  return loss.data[0]

학습 결과를 기반으로 신경망과 편향을 조정하자.

In [ ]:
def optimize(learning_rate):
  w.data -= learning_rate * w.grad.data
  b.data -= learning_rate * b.grad.data